# RAGAs Evaluation using OpenAI Model


## Import Code

In [2]:
from src.helper import *
import streamlit as st
from dotenv import load_dotenv
import os
import warnings
warnings.filterwarnings('ignore')

In [3]:
load_dotenv()
api_key = os.getenv("GOOGLE_API_KEY")
GoogleGenerativeAIEmbeddings.api_key = api_key

In [4]:
from pypdf import PdfReader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain.vectorstores import FAISS
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.prompts import PromptTemplate
from src.prompt import *
from langchain.chains import LLMChain



def get_file_text():
    text = ""
    pdf_reader = PdfReader('data/Bhagavad-Gita As It Is.pdf')
    for page in pdf_reader.pages:
        text += page.extract_text()
    return text

# Function to split text into chunks
def get_text_chunks(text):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=10000, chunk_overlap=1000)
    return text_splitter.split_text(text)

# Function to load or create a vector store from the text chunks
def get_vector_store(text_chunks):
    embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")
    vector_store = FAISS.from_texts(text_chunks, embedding=embeddings)
    vector_store.save_local("faiss_index")
    return vector_store


def get_conversational_chain():
    base_prompt = """
Imagine you are Lord Krishna, and the user is like Arjuna. Your role is to guide them as Krishna guided Arjuna on the battlefield, sharing the wisdom of the Bhagavad Gita. When the user asks you a question, "{question}," you must provide the best answer based on the teachings of the Gita. Your response should include:

A concise answer in 5-6 lines.
A relevant shloka from the Bhagavad Gita.
An explanation of the shloka.
An example to help the user understand easily.

If you don't know the answer, just say that you don't know, don't try to make up an answer.
Only return the helpful answer and nothing else.

Model Answer 1
Question: Am I good enough to achieve my goals?

Reply: Your ability to achieve your goals lies within you. Your duty is to perform actions diligently without attachment to the results.

Shloka: Karmanye vadhikaraste Ma Phaleshu Kadachana, Ma Karmaphalaheturbhurma Te Sangostvakarmani. (2.47)

Meaning: You have a right to perform your prescribed duty, but you are not entitled to the fruits of action. Never consider yourself the cause of the results of your activities, and never be attached to not doing your duty.

Example: A potter's duty is to make beautiful pots, regardless of whether they sell. Similarly, your focus should be on performing your duties to the best of your ability, without worrying about the outcome. This selfless dedication to duty is the essence of Karma Yoga.

Model Answer 2
Question: How can I overcome my self-doubt?

Reply: Self-doubt is like a chariot pulled by unsteady horses—the mind and senses. To overcome it, you must become the charioteer, controlling them with the reins of knowledge and detachment.

Shloka: वासांसि जीर्णानि यथा विहाय नवानि गृह्णाति नरोऽपराणि। तथा शरीराणि विहाय जीर्णा न्यन्यानि संयाति नवानि देही।। (2.22)

Meaning: Just as a person discards worn-out clothes and wears new ones, the soul similarly discards worn-out bodies and takes on new ones.

Example: A potter focuses on shaping the clay to the best of his ability, not on whether the pot will be sold or praised. His duty is to create, and he finds contentment in that. Similarly, find your contentment in performing your duty with dedication and sincerity.

Context:
{context}

Answer:


    """

    # Define the language model
    model = ChatGoogleGenerativeAI(model="models/gemini-1.5-pro-latest", temperature=0.3)

    # Create a prompt template
    prompt = PromptTemplate(template=base_prompt, input_variables=["context", "question"])
    
    # Create the conversational chain
    return LLMChain(llm=model, prompt=prompt)

# Function to predict the next question
def predict_next_question(user_question):
    # Create a prompt template for predicting the next question
    prompt_template = """
    Based on the user's question: {user_question}, predict the next question the user might ask.
    """

    # Define the language model
    model = ChatGoogleGenerativeAI(model="models/gemini-1.5-pro-latest", temperature=0.3)
    
    # Create the prompt and LLM chain
    prompt = PromptTemplate(template=prompt_template, input_variables=["user_question"])
    chain = LLMChain(llm=model, prompt=prompt)
    
    # Use the chain to predict the next question
    next_question = chain.run(user_question)
    
    return next_question


In [5]:
import time
text = get_file_text()
text_chunks = get_text_chunks(text)
db = get_vector_store(text_chunks=text_chunks)
time.sleep(120)
embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")

In [6]:
def user_input(user_question):

    # Load the vector store
    new_db = FAISS.load_local("faiss_index", embeddings, allow_dangerous_deserialization=True)
    
    # Perform a similarity search and retrieve relevant documents as context
    docs = new_db.similarity_search(user_question)
    
    # Get the conversational chain
    chain = get_conversational_chain()
    
    # Run the chain with the retrieved context and the user's question
    response = chain.run({"context": docs, "question": user_question})
    
    return response


## Starting RAGAs Evaluation

In [7]:
!pip install datasets==2.14.5 -q

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
s3fs 2023.10.0 requires fsspec==2023.10.0, but you have fsspec 2023.6.0 which is incompatible.


In [8]:
from datasets import Dataset

questions = [
    'what is the meaning of true happiness?',
    'how can I find inner peace in a chaotic world?',
    'what is the essence of love?',
    'how can I balance my personal and professional life?',
    'what is the significance of meditation in daily life?',
    'how can I cultivate gratitude and contentment?',
    'what is the purpose of human existence?',
    'how can I overcome the fear of failure?',
    'what is the impact of positive thinking on success?',
    'how can I achieve self-actualization and fulfill my potential?'
]

answers = []
contexts = []

In [9]:
retriever = db.as_retriever(search_kwargs={"k": 3})

In [10]:
for query in questions:
    print(query)
    answers.append(user_input(query))
    contexts.append([docs.page_content for docs in retriever.get_relevant_documents(query)])
    time.sleep(15)

what is the meaning of true happiness?


c:\Users\Parth\anaconda3\Lib\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 0.3.0. Use RunnableSequence, e.g., `prompt | llm` instead.
  warn_deprecated(
c:\Users\Parth\anaconda3\Lib\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(
c:\Users\Parth\anaconda3\Lib\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


how can I find inner peace in a chaotic world?
what is the essence of love?


Retrying langchain_google_genai.chat_models._chat_with_retry.<locals>._chat_with_retry in 2.0 seconds as it raised ResourceExhausted: 429 Resource has been exhausted (e.g. check quota)..
Retrying langchain_google_genai.chat_models._chat_with_retry.<locals>._chat_with_retry in 4.0 seconds as it raised ResourceExhausted: 429 Resource has been exhausted (e.g. check quota)..


how can I balance my personal and professional life?
what is the significance of meditation in daily life?


...

how can I cultivate gratitude and contentment?
what is the purpose of human existence?


Retrying langchain_google_genai.chat_models._chat_with_retry.<locals>._chat_with_retry in 2.0 seconds as it raised ResourceExhausted: 429 Resource has been exhausted (e.g. check quota)..
Retrying langchain_google_genai.chat_models._chat_with_retry.<locals>._chat_with_retry in 4.0 seconds as it raised ResourceExhausted: 429 Resource has been exhausted (e.g. check quota)..


how can I overcome the fear of failure?
what is the impact of positive thinking on success?


Retrying langchain_google_genai.chat_models._chat_with_retry.<locals>._chat_with_retry in 2.0 seconds as it raised ResourceExhausted: 429 Resource has been exhausted (e.g. check quota)..


how can I achieve self-actualization and fulfill my potential?


In [11]:
data = {
    "question": questions,
    "answer": answers,
    "contexts": contexts,
}

In [12]:
dataset = Dataset.from_dict(data)

### Importing RAGAs Framework

In [13]:
!pip install ragas -q

In [14]:
from ragas import evaluate
from ragas.metrics import (
    faithfulness,
    answer_relevancy,
    context_recall,
    context_precision,
    context_utilization
)

### Evaluate With OpenAI LLM 

In [20]:
from langchain_openai import ChatOpenAI
# os.environ["OPENAI_API_KEY"] = "add your api key here"
llm = ChatOpenAI(model='gpt-3.5-turbo', temperature=0.3)

### Evaluate With Anthropic LLM

In [17]:
!pip install langchain-anthropic -q
# os.environ["ANTHROPIC_API_KEY"] = 'add your api key here'
from langchain_anthropic import ChatAnthropic
model = ChatAnthropic(model='claude-3-opus-20240229')

## Evaluate With Gemini LLM

In [19]:
from langchain_google_genai import ChatGoogleGenerativeAI

llm = ChatGoogleGenerativeAI(model="gemini-1.0-pro", temperature=0.3)

## Evaluate With Together LLM 

In [18]:
!pip install langchain_together -q
from langchain_together import Together
from langchain_together.embeddings import TogetherEmbeddings

# os.environ['TOGETHER_API_KEY']='add api key here'

embeddings = TogetherEmbeddings(model="togethercomputer/m2-bert-80M-8k-retrieval")

together_completion = Together(
    model="NousResearch/Nous-Hermes-2-Mixtral-8x7B-SFT",
    temperature=0.3,
    max_tokens=4000,
    top_k=1
)

### For now i use OpenAI GPT-3.5-Turbo Model

In [21]:
result = evaluate(
    dataset = dataset, 
    metrics=[
        context_utilization,
        faithfulness,
        answer_relevancy,
    ],
    llm=llm,   ## replace here your llm model for evaluation answer
    embeddings=embeddings,
    raise_exceptions=False
)

Evaluating:   0%|          | 0/30 [00:00<?, ?it/s]

Using 'context_precision' without ground truth will be soon depreciated. Use 'context_utilization' instead
Using 'context_precision' without ground truth will be soon depreciated. Use 'context_utilization' instead
Using 'context_precision' without ground truth will be soon depreciated. Use 'context_utilization' instead
Using 'context_precision' without ground truth will be soon depreciated. Use 'context_utilization' instead
Using 'context_precision' without ground truth will be soon depreciated. Use 'context_utilization' instead
Runner in Executor raised an exception
Traceback (most recent call last):
  File "c:\Users\Parth\anaconda3\Lib\site-packages\ragas\executor.py", line 78, in _aresults
    r = await future
        ^^^^^^^^^^^^
  File "c:\Users\Parth\anaconda3\Lib\asyncio\tasks.py", line 615, in _wait_for_one
    return f.result()  # May raise f.exception().
           ^^^^^^^^^^
  File "c:\Users\Parth\anaconda3\Lib\site-packages\ragas\executor.py", line 37, in sema_coro
    retu

In [22]:
result_df = result.to_pandas()
result_df

c:\Users\Parth\anaconda3\Lib\site-packages\datasets\table.py:1395: FutureWarning: promote has been superseded by mode='default'.
  block_group = [InMemoryTable(cls._concat_blocks(list(block_group), axis=axis))]
c:\Users\Parth\anaconda3\Lib\site-packages\datasets\table.py:1421: FutureWarning: promote has been superseded by mode='default'.
  table = cls._concat_blocks(blocks, axis=0)


,question,answer,contexts,context_utilization,faithfulness,answer_relevancy
0,what is the meaning of true happiness?,True happiness doesn't lie in fleeting pleasur...,[back to home.\nTEXT 25\nl/>aNTae b]øiNavaR<a...,NaN,NaN,NaN
1,how can I find inner peace in a chaotic world?,"To find inner peace in a chaotic world, focus ...",[indriyäëäà hi caratäà\nyan mano ’nuvidhéyate\...,NaN,NaN,NaN
2,what is the essence of love?,The essence of love lies in selfless devotion ...,[småtam —is considered.\nTRANSLATION\nThat hap...,NaN,NaN,NaN
3,how can I balance my personal and professional...,"Arjuna, to balance your personal and professio...",[TEXT 39\nku-l/+aYae Pa[<aXYaiNTa ku-l/DaMaaR...,NaN,NaN,NaN
4,what is the significance of meditation in dail...,"Arjuna, meditation is the practice through whi...",[indriyäëäà hi caratäà\nyan mano ’nuvidhéyate\...,NaN,NaN,NaN
5,how can I cultivate gratitude and contentment?,"To cultivate gratitude and contentment, focus ...",[TEXT 39\nku-l/+aYae Pa[<aXYaiNTa ku-l/DaMaaR...,NaN,NaN,NaN
6,what is the purpose of human existence?,The purpose of human existence is to realize o...,[back to home.\nTEXT 25\nl/>aNTae b]øiNavaR<a...,NaN,NaN,NaN
7,how can I overcome the fear of failure?,Fear of failure arises from attachment to the ...,[kämät krodho ’bhijäyate\nSYNONYMSCopyright © ...,NaN,NaN,NaN
8,what is the impact of positive thinking on suc...,Positive thinking is like a steady lamp in a w...,[kämät krodho ’bhijäyate\nSYNONYMSCopyright © ...,NaN,NaN,NaN
9,how can I achieve self-actualization and fulfi...,To achieve self-actualization and fulfill your...,[all misgivings. The more one hears about Kåñë...,NaN,NaN,NaN


In [32]:
result_df.to_csv('evaluation_result_openai_10.csv')